In [ ]:
import asyncio
from playwright.async_api import async_playwright
import openai  # or Gemini wrapper depending on access
import chromadb
from chromadb.config import Settings
from uuid import uuid4
from typing import Dict, Any
import os
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import random

In [ ]:
CHROMA_PATH = "./chroma_db"
COLLECTION_NAME = "book_versions"
FEEDBACK_LOG = "retrieval_feedback.json"

In [ ]:
client = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet", persist_directory=CHROMA_PATH))
collection = client.get_or_create_collection(COLLECTION_NAME)

In [ ]:
if not os.path.exists(FEEDBACK_LOG):
    with open(FEEDBACK_LOG, "w") as f:
        json.dump({}, f)

In [ ]:
async def fetch_content_and_screenshot(url: str) -> str:
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        content = await page.inner_text("body")
        await page.screenshot(path="chapter_screenshot.png", full_page=True)
        await browser.close()
    return content

In [ ]:
def ai_writer_agent(text: str) -> str:
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You're an expert fiction writer rephrasing content engagingly."},
            {"role": "user", "content": text}
        ]
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
def ai_reviewer_agent(text: str) -> str:
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You're an editor refining tone, clarity, and flow."},
            {"role": "user", "content": text}
        ]
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
def human_feedback_loop(text: str, iterations: int = 2) -> str:
    for i in range(iterations):
        print(f"\n--- Human Review Round {i+1} ---")
        print("Preview:")
        print("=" * 80)
        print(text[:1000], "...\n")
        print("=" * 80)
        choice = input("Accept this version? (y = yes / n = suggest edit): ").strip().lower()
        if choice == "n":
            user_edit = input("Enter your suggestion: ")
            text += f"\n[Editor Suggestion Round {i+1}]: {user_edit}"
        else:
            print("✅ Approved by human editor.")
            break
    return text

In [ ]:
def save_version(chapter_id: str, text: str):
    collection.add(
        documents=[text],
        ids=[chapter_id],
        metadatas=[{"source": "spun_chapter", "version": str(uuid4())}]
    )

In [ ]:
def get_embedding(text: str) -> list:
    response = openai.Embedding.create(input=[text], model="text-embedding-ada-002")
    return response["data"][0]["embedding"]

In [ ]:
def rl_search(query: str) -> str:
    query_vec = get_embedding(query)
    results = collection.get(include=["documents", "metadatas", "embeddings"])

    if not results["documents"]:
        return "No documents found."

    doc_vectors = results["embeddings"]
    similarities = cosine_similarity([query_vec], doc_vectors)[0]

In [ ]:
 epsilon = 0.2
    if random.random() < epsilon:
        top_idx = random.randint(0, len(similarities) - 1)
        print("🎲 Exploring random result...")
    else:
        top_idx = int(np.argmax(similarities))
        print("🎯 Exploiting best result...")

    best_doc = results["documents"][top_idx][0]
    best_id = results["ids"][top_idx]

    log_feedback(query, best_id, similarities[top_idx])
    return best_doc

In [ ]:
ef log_feedback(query: str, doc_id: str, score: float):
    with open(FEEDBACK_LOG, "r+") as f:
        data = json.load(f)
        entry = { "query": query, "doc_id": doc_id, "score": score }
        data.setdefault("logs", []).append(entry)
        f.seek(0)
        json.dump(data, f, indent=2)

In [ ]:
def run_workflow(url: str, chapter_id: str):
    content = asyncio.run(fetch_content_and_screenshot(url))
    spun = ai_writer_agent(content)
    reviewed = ai_reviewer_agent(spun)
    final = human_feedback_loop(reviewed)
    save_version(chapter_id, final)
    print("\n✅ Chapter processed and saved.\n")

In [ ]:
if __name__ == "__main__":
    run_workflow(
        url="https://en.wikisource.org/wiki/The_Gates_of_Morning/Book_1/Chapter_1",
        chapter_id="gates_morning_book1_ch1"
    )

In [ ]:
 print("🔍 Retrieving Final Version:")
    print(rl_search("The Gates of Morning Chapter 1"))
